In [1]:
import argparse
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd


import networkx as nx
from typing import Union

import random
import pickle

from MatrixVectorizer import *
from dataloaders import NoisyDataset
from model import *
from preprocessing import *
from test import *

In [2]:
# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device("cpu")

In [3]:
# load csvs as numpy
lr_data_path = './data/lr_train.csv'
hr_data_path = './data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data
lr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data],
                                        dtype=torch.float32)
hr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data],
                                        dtype=torch.float32)


/tmp/ipykernel_8928/3687639064.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  lr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data],


In [4]:
train_data = NoisyDataset(lr_train_data_vectorized, hr_train_data_vectorized, noise_level=0.5)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

In [5]:
num_splt = 3
epochs = 150
lr = 0.00005 # try [0.0001, 0.0005, 0.00001, 0.00005]
lmbda = 17 # should be around 15-20
lamdba_topo = 0.0005 # should be around 0.0001-0.001
lr_dim = 160
hr_dim = 320
hidden_dim = 320 # try smaller and larger - [160-512]
padding = 26
dropout = 0.2 # try [0., 0.1, 0.2, 0.3]


args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lamdba_topo = lamdba_topo
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [7]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [8]:
criterion = nn.L1Loss()

In [9]:
def generate_submission_csv(model, data_path='./data/lr_test.csv', filename='submission.csv'):
    lr_test_data = pd.read_csv(data_path, delimiter=',').to_numpy()
    lr_test_data[lr_test_data < 0] = 0
    np.nan_to_num(lr_test_data, copy=False)
    lr_test_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_test_data])
    model.eval()
    preds = []
    for lr in lr_test_data_vectorized:      
        lr = torch.from_numpy(lr).type(torch.FloatTensor)
        
        model_outputs, _, _, _ = model(lr)
        model_outputs  = unpad(model_outputs, args.padding)
        preds.append(MatrixVectorizer.vectorize(model_outputs.detach().numpy()))

    r = np.hstack(preds)
    meltedDF = r.flatten()
    n = meltedDF.shape[0]
    df = pd.DataFrame({'ID': np.arange(1, n+1),
                    'Predicted': meltedDF})
    df.to_csv(filename, index=False)

In [9]:

def train(model, train_data_loader, optimizer, args): 
  
  all_epochs_loss = []
  all_epochs_error = []
  all_epochs_topoloss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []
    epoch_topo = []

    model.train()
    for lr, hr in train_data_loader: 
      lr.to(device)   
      hr.to(device)  
      lr = lr.reshape(160, 160)
      hr = hr.reshape(268, 268)

      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs  = unpad(model_outputs, args.padding)

      padded_hr = pad_HR_adj(hr,args.padding)
      _, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
      topo = compute_topological_MAE_loss(hr, model_outputs)
      
      loss += args.lamdba_topo * topo

      error = criterion(model_outputs, hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      epoch_topo.append(topo.item())
      
  
    model.eval()
    if (epoch + 1) % 100 == 0:
      filename = f'{epoch+1}epoch-model.sav'
      pickle.dump(model, open(filename, 'wb'))
      # generate_submission_csv(model, filename=f'{epoch+1}epoch-model.csv')
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error),
          "Topo: ", np.mean(epoch_topo))
    all_epochs_loss.append(np.mean(epoch_loss))
    all_epochs_error.append(np.mean(epoch_error))
    all_epochs_topoloss.append(np.mean(epoch_topo))
  df = pd.DataFrame({'Epoch': np.arange(1, no_epochs+1),
                  'Total Loss': all_epochs_loss,
                  'Error': all_epochs_error,
                  'Topological loss': all_epochs_topoloss,
                  })
  df.to_csv('model-losses.csv', index=False)
  

# Final Model & Kaggle Submission

In [10]:
#final train
final_model = GSRNet(ks, args)
optimizer = optim.Adam(final_model.parameters(), lr=args.lr)

final_model.to(device)

train(final_model, train_data_loader, optimizer, args)

Epoch:  1 Loss:  0.4603491131060138 Error:  0.20874980636342558 Topo:  36.38929357357368
Epoch:  2 Loss:  0.31078401975289077 Error:  0.18189827383992202 Topo:  20.48308480999427
Epoch:  3 Loss:  0.30281120181797505 Error:  0.178619987325754 Topo:  19.349652804300458
Epoch:  4 Loss:  0.29687833732473634 Error:  0.17685522817209096 Topo:  19.053390000394717
Epoch:  5 Loss:  0.29146927898515484 Error:  0.17582334578037262 Topo:  18.929457476039133
Epoch:  6 Loss:  0.2861098129592256 Error:  0.17474247675812887 Topo:  18.69257019522661
Epoch:  7 Loss:  0.2810191350604246 Error:  0.17396613295206767 Topo:  18.826101759950557
Epoch:  8 Loss:  0.27654760424605385 Error:  0.1733112237053717 Topo:  18.731812168738085
Epoch:  9 Loss:  0.2726511842833308 Error:  0.17260693754264694 Topo:  18.763306115201846
Epoch:  10 Loss:  0.2684864010104162 Error:  0.1717383844588331 Topo:  18.833300008031422
Epoch:  11 Loss:  0.2648830295144441 Error:  0.17108892502185113 Topo:  18.707007956362055
Epoch:  12

NameError: name 'generate_submission_csv' is not defined

In [12]:
import pickle
filename = 'final-model.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [13]:
generate_submission_csv(final_model)